In [1]:
!pip install -q google-generativeai chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.8 MB/s eta

In [4]:
import os
import re
from typing import List, Dict, Tuple
import google.generativeai as genai
import chromadb
from chromadb.config import Settings

# ==================== CONFIGURATION ====================

class RAGConfig:
    """Configuration for RAG system"""
    GEMINI_MODEL = "gemini-2.5-flash-lite"
    EMBEDDING_MODEL = "models/text-embedding-004"
    CHUNK_SIZE = 500
    CHUNK_OVERLAP = 50 # Chunk1 and Chunk 2 will have 50 char similar
    TOP_K_RESULTS = 3
    COLLECTION_NAME = "rag_documents"

from google.colab import userdata
GEMINI_API_KEY = userdata.get("key_1")
# Set your API key  # Replace with your actual API key
genai.configure(api_key=GEMINI_API_KEY)

In [5]:
class CustomTextLoader:
    """Custom loader for reading .txt files"""

    @staticmethod
    def load_file(file_path: str) -> str:
        """Load text from a single .txt file"""
        if not file_path.endswith('.txt'):
            raise ValueError("Only .txt files are supported")

        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        return content

In [9]:
@staticmethod
def load_directory(directory_path: str) -> List[Dict[str, str]]:
        """Load all .txt files from a directory"""
        documents = []

        for filename in os.listdir(directory_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(directory_path, filename)
                content = CustomTextLoader.load_file(file_path)
                documents.append({
                    'content': content,
                    'source': filename,
                    'path': file_path
                })

        return documents

In [10]:
# ==================== CUSTOM TEXT SPLITTER ====================

class CustomTextSplitter:
    """Custom text splitter with overlap support"""

    def __init__(self, chunk_size: int = 500, chunk_overlap: int = 50):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    def split_text(self, text: str) -> List[str]:
        """Split text into chunks with overlap"""
        # Clean the text
        text = self._clean_text(text)

        # Split by sentences first for better semantic boundaries
        sentences = self._split_into_sentences(text)

        chunks = []
        current_chunk = []
        current_length = 0

        for sentence in sentences:
            sentence_length = len(sentence)

            if current_length + sentence_length > self.chunk_size and current_chunk:
                # Save current chunk
                chunks.append(' '.join(current_chunk))

                # Create overlap by keeping last few sentences
                overlap_text = ' '.join(current_chunk)
                overlap_sentences = []
                overlap_length = 0

                for s in reversed(current_chunk):
                    if overlap_length + len(s) <= self.chunk_overlap:
                        overlap_sentences.insert(0, s)
                        overlap_length += len(s)
                    else:
                        break

                current_chunk = overlap_sentences
                current_length = overlap_length

            current_chunk.append(sentence)
            current_length += sentence_length

        # Add the last chunk
        if current_chunk:
            chunks.append(' '.join(current_chunk))

        return chunks

    @staticmethod
    def _clean_text(text: str) -> str:
        """Clean and normalize text"""
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        # Remove special characters but keep punctuation
        text = re.sub(r'[^\w\s.,!?;:\-\'\"()]', '', text)
        return text.strip()

    @staticmethod
    def _split_into_sentences(text: str) -> List[str]:
        """Split text into sentences"""
        # Simple sentence splitter
        sentences = re.split(r'(?<=[.!?])\s+', text)
        return [s.strip() for s in sentences if s.strip()]

In [11]:

# ==================== EMBEDDING MANAGER ====================

class EmbeddingManager:
    """Manage embeddings using Gemini's embedding model"""

    def __init__(self, model_name: str = RAGConfig.EMBEDDING_MODEL):
        self.model_name = model_name

    def embed_text(self, text: str) -> List[float]:
        """Generate embedding for a single text"""
        result = genai.embed_content(
            model=self.model_name,
            content=text,
            task_type="retrieval_document"
        )
        return result['embedding']

    def embed_query(self, query: str) -> List[float]:
        """Generate embedding for a query"""
        result = genai.embed_content(
            model=self.model_name,
            content=query,
            task_type="retrieval_query"
        )
        return result['embedding']

    def embed_batch(self, texts: List[str]) -> List[List[float]]:
        """Generate embeddings for multiple texts"""
        embeddings = []
        for text in texts:
            embedding = self.embed_text(text)
            embeddings.append(embedding)
        return embeddings



In [12]:
# ==================== EMBEDDING MANAGER ====================

class EmbeddingManager:
    """Manage embeddings using Gemini's embedding model"""

    def __init__(self, model_name: str = RAGConfig.EMBEDDING_MODEL):
        self.model_name = model_name

    def embed_text(self, text: str) -> List[float]:
        """Generate embedding for a single text"""
        result = genai.embed_content(
            model=self.model_name,
            content=text,
            task_type="retrieval_document"
        )
        return result['embedding']

    def embed_query(self, query: str) -> List[float]:
        """Generate embedding for a query"""
        result = genai.embed_content(
            model=self.model_name,
            content=query,
            task_type="retrieval_query"
        )
        return result['embedding']

    def embed_batch(self, texts: List[str]) -> List[List[float]]:
        """Generate embeddings for multiple texts"""
        embeddings = []
        for text in texts:
            embedding = self.embed_text(text)
            embeddings.append(embedding)
        return embeddings


In [13]:
# ==================== VECTOR STORE ====================
class VectorStore:
    """ChromaDB vector store manager"""

    def __init__(self, collection_name: str = RAGConfig.COLLECTION_NAME):
        self.client = chromadb.Client(Settings(
            anonymized_telemetry=False,
            allow_reset=True
        ))
        self.collection_name = collection_name
        self.collection = None
        self.embedding_manager = EmbeddingManager()

    def create_collection(self, reset: bool = False):
        """Create or get collection"""
        if reset:
            try:
                self.client.delete_collection(name=self.collection_name)
            except:
                pass

        self.collection = self.client.get_or_create_collection(
            name=self.collection_name,
            metadata={"description": "RAG document embeddings"}
        )

    def add_documents(self, chunks: List[str], metadatas: List[Dict]):
        """Add document chunks to the vector store"""
        if not self.collection:
            self.create_collection()

        print(f"Generating embeddings for {len(chunks)} chunks...")
        embeddings = self.embedding_manager.embed_batch(chunks)

        # Generate IDs
        ids = [f"chunk_{i}" for i in range(len(chunks))]

        print(f"Adding {len(chunks)} chunks to ChromaDB...")
        self.collection.add(
            embeddings=embeddings,
            documents=chunks,
            metadatas=metadatas,
            ids=ids
        )
        print("✓ Documents added successfully!")

    def query(self, query_text: str, top_k: int = RAGConfig.TOP_K_RESULTS) -> Dict:
        """Query the vector store"""
        if not self.collection:
            raise ValueError("Collection not initialized")

        # Generate query embedding
        query_embedding = self.embedding_manager.embed_query(query_text)

        # Search
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k
        )

        return results

In [14]:
# ==================== RAG SYSTEM ====================

class RAGSystem:
    """Complete RAG system"""

    def __init__(self):
        self.vector_store = VectorStore()
        self.text_splitter = CustomTextSplitter(
            chunk_size=RAGConfig.CHUNK_SIZE,
            chunk_overlap=RAGConfig.CHUNK_OVERLAP
        )
        self.model = genai.GenerativeModel(RAGConfig.GEMINI_MODEL)

    def ingest_documents(self, file_path: str = None, directory_path: str = None, reset: bool = True):
        """Ingest documents from file or directory"""
        loader = CustomTextLoader()

        # Load documents
        if file_path:
            content = loader.load_file(file_path)
            documents = [{'content': content, 'source': os.path.basename(file_path)}]
        elif directory_path:
            documents = loader.load_directory(directory_path)
        else:
            raise ValueError("Either file_path or directory_path must be provided")

        print(f"Loaded {len(documents)} document(s)")

        # Split documents into chunks
        all_chunks = []
        all_metadata = []

        for doc in documents:
            chunks = self.text_splitter.split_text(doc['content'])
            print(f"Split {doc['source']} into {len(chunks)} chunks")

            for i, chunk in enumerate(chunks):
                all_chunks.append(chunk)
                all_metadata.append({
                    'source': doc['source'],
                    'chunk_id': i,
                    'total_chunks': len(chunks)
                })

        # Create collection and add documents
        self.vector_store.create_collection(reset=reset)
        self.vector_store.add_documents(all_chunks, all_metadata)

        print(f"\n✓ Ingestion complete! Total chunks: {len(all_chunks)}")

    def query(self, question: str, top_k: int = RAGConfig.TOP_K_RESULTS) -> Tuple[str, List[Dict]]:
        """Query the RAG system"""
        # Retrieve relevant documents
        results = self.vector_store.query(question, top_k=top_k)

        # Extract context
        contexts = results['documents'][0]
        metadatas = results['metadatas'][0]

        # Build context string
        context_str = "\n\n".join([
            f"[Source: {meta['source']}, Chunk {meta['chunk_id']+1}/{meta['total_chunks']}]\n{doc}"
            for doc, meta in zip(contexts, metadatas)
        ])

        # Create prompt
        prompt = f"""Answer the question based on the context provided.
        If the answer cannot be found in the context,
        say "I cannot answer this based on the provided context."

Context:
{context_str}

Question: {question}

Answer:"""

        # Generate response
        response = self.model.generate_content(prompt)

        # Prepare source information
        sources = [
            {
                'source': meta['source'],
                'chunk_id': meta['chunk_id'],
                'text': doc[:200] + "..."
            }
            for doc, meta in zip(contexts, metadatas)
        ]

        return response.text, sources


In [15]:

# ==================== INTERACTIVE MODE ====================

def interactive_mode():
    """Run RAG system in interactive mode"""
    print(" Starting RAG System in Interactive Mode...")
    rag = RAGSystem()

    # Get file path from user
    file_path = input("\nEnter path to .txt file (or press Enter to use sample): ").strip()

    if not file_path:
        # Create and use sample file
        print("Using sample document...")
        with open('sample_ai.txt', 'w', encoding='utf-8') as f:
            f.write("""
            Artificial Intelligence is revolutionizing technology. Machine learning enables
            computers to learn from data. Deep learning uses neural networks for complex tasks.
            """)
        file_path = 'sample_ai.txt'

    # Ingest documents
    rag.ingest_documents(file_path=file_path)

    # Interactive query loop
    print("\n" + "="*70)
    print(" RAG System Ready! Type 'quit' or 'exit' to stop.")
    print("="*70)

    while True:
        question = input("\n❓ Your question: ").strip()

        if question.lower() in ['quit', 'exit', 'q']:
            print(" Goodbye!")
            break

        if not question:
            continue

        try:
            answer, sources = rag.query(question)
            print(f"\n Answer:\n{answer}\n")
            print("📄 Sources:")
            for i, source in enumerate(sources, 1):
                print(f"  {i}. {source['source']} (Chunk {source['chunk_id']+1})")
        except Exception as e:
            print(f" Error: {str(e)}")

# Uncomment to run interactive mode:
interactive_mode()

 Starting RAG System in Interactive Mode...

Enter path to .txt file (or press Enter to use sample): 
Using sample document...
Loaded 1 document(s)
Split sample_ai.txt into 1 chunks
Generating embeddings for 1 chunks...
Adding 1 chunks to ChromaDB...
✓ Documents added successfully!

✓ Ingestion complete! Total chunks: 1

 RAG System Ready! Type 'quit' or 'exit' to stop.

❓ Your question: what is ai

 Answer:
Artificial Intelligence is revolutionizing technology.

📄 Sources:
  1. sample_ai.txt (Chunk 1)

❓ Your question: who is virat

 Answer:
I cannot answer this based on the provided context.

📄 Sources:
  1. sample_ai.txt (Chunk 1)

❓ Your question: what is computer vision

 Answer:
I cannot answer this based on the provided context.

📄 Sources:
  1. sample_ai.txt (Chunk 1)

❓ Your question: what is nlp

 Answer:
I cannot answer this based on the provided context.

📄 Sources:
  1. sample_ai.txt (Chunk 1)

❓ Your question: quit
 Goodbye!


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
# ==================== EXAMPLE USAGE ====================

def main():
    """Example usage of the RAG system"""

    # Initialize RAG system
    print("🚀 Initializing RAG System...")
    rag = RAGSystem()

    # Example 1: Create a sample text file
    print("\n📝 Creating sample document...")
    sample_text = """
    Artificial Intelligence (AI) is transforming the world. Machine learning, a subset of AI,
    enables computers to learn from data without explicit programming. Deep learning uses neural
    networks with multiple layers to process complex patterns.

    Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
    It powers applications like chatbots, translation services, and sentiment analysis.

    Computer vision is another important AI field that enables machines to interpret visual information.
    It's used in facial recognition, autonomous vehicles, and medical imaging.

    The future of AI includes advancements in quantum computing, edge AI, and explainable AI systems.
    """

    with open('sample_ai.txt', 'w', encoding='utf-8') as f:
        f.write(sample_text)

    # Example 2: Ingest the document
    print("\n📚 Ingesting documents...")
    rag.ingest_documents(file_path='sample_ai.txt')

    # Example 3: Query the system
    print("\n" + "="*70)
    print("💬 RAG SYSTEM READY - Ask questions!")
    print("="*70)

    questions = [
        "What is machine learning?",
        "What are the applications of NLP?",
        "What is the future of AI?"
    ]